This is an implementation of the artistic neural network as described in this [paper](https://arxiv.org/abs/1508.06576). VGG pretrained model weights can be downloaded from [here](http://www.vlfeat.org/matconvnet/models/). Use imagenet-vgg-verydeep-19.mat and imagenet-vgg-verydeep-16.mat

In [3]:
import os
import sys
import numpy as np
import scipy.io
import scipy.misc
import tensorflow as tf

In [6]:
class VGG(object):
    
    def __init__(self):
        self.model = None
    
    def factory(name, model_path):
        """
        The factory to create the corresponding model we will use
        Available names include "VGG16" and "VGG19"
        """
        if name == "VGG16": return VGG16(model_path)
        if name == "VGG19": return VGG19(model_path)
        
    factory = staticmethod(factory)
    
    def load_model(self):
        raise NotImplementedError("Abstract method")
    
    def content_loss(self):
        raise NotImplementedError("Abstract method")
    
    def style_loss(self):
        raise NotImplementedError("Abstract method")
        
    def emit(self):
        return self.model

In [4]:
class VGG19(VGG):
    
    def __init__(self, model_path):
        self.path  = model_path
        self.model = load_model()
    
    def load_model(self):
        vgg_layers = scipy.io.loadmat(self.path)['layers']
        num_layers = len(vgg_layers[0])
        

In [44]:
vgg = scipy.io.loadmat("models/imagenet-vgg-verydeep-16.mat")

In [48]:
print vgg['layers'][0][0][0][0][1]

[u'conv']


In [52]:
print len(vgg['layers'][0])

37
